In [19]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('restaurant_reviews.csv')
sc = SparkContext(conf=conf)
sc

23/11/15 14:13:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local appName=restaurant_reviews.csv>

# Reduce

- RDD.reduce(<task>)
-

In [20]:
from operator import add

In [21]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5]).reduce(add) # add : lambda x, y : x + y
sample_rdd

15

# 파티션에 따라 결과물이 달라진다.

- 분산된 파티션들의 연산과 합치는 부분을 나눠서 생각해야 함
    lambda x, y : (x * 2) + y

In [22]:
# 파티션이 1개인 경우
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y : (x * 2) + y)

26

In [23]:
# 파티션이 2개인 경우
sc.parallelize([1,2,3,4],2).reduce(lambda x, y : (x * 2) + y)

18

In [24]:
# 파티션이 3개인 경우
sc.parallelize([1,2,3,4],3).reduce(lambda x, y : (x * 2) + y)

18

In [25]:
# 파티션이 4개인 경우
sc.parallelize([1,2,3,4],4).reduce(lambda x, y : (x * 2) + y)

26

# Fold
- RDD.fold(zeroValue, <task>
- reduce와 비슷하지만, 시작값을 zeroValue에 넣어놓고 reduce 할 수 있다.   

In [26]:
rdd = sc.parallelize([2, 3, 4],4)

In [27]:
print(rdd.reduce(lambda x, y : (x * y))) # 2 * 3 = 6, 6 * 4 = 24
print(rdd.fold(1, lambda x, y : (x * y))) 

24
24


In [28]:
print(rdd.reduce(lambda x, y : x + y))

9


In [29]:
print(rdd.fold(1, lambda x, y : x + y))

14


# Aggregate(Action)

- RDD.aggregate(zeroValue, seq0p, comb0p
        - zeroValue : 각 파티션에서 누적할 시작값
        - seq0p : 타입 변경 함수, 파티션 내부에서 사용할 task
        - comb0p : 합치는 함수, 파티션 끼리 사용할 task

In [34]:
rdd = sc.parallelize([1,2,3,4,],2)

In [37]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1)) # 파티션 내의 연산
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1])) # 파티션의 모든 결과를 최종 연산

In [38]:
rdd.aggregate((0, 0), seqOp, combOp)

(10, 4)

In [39]:
sc.stop()